In [ ]:
# This model is designed to perform automated segmentation and classification of sediment features.
# It focuses on identifying specific particle properties and categorizing them into distinct classes.
# The pipeline starts by using a U-Net-based convolutional neural network to segment images, separating grains from the surrounding matrix.
# Once the grains are segmented, the model extracts a range of geometric and shape-related properties for each particle.
# These properties include roundness, elongation, and rugosity, as well as measurements like perimeter, Feret’s diameter, and overall particle size.
# The classification stage employs several machine learning algorithms, including support vector machines (SVM), logistic regression, and Naive Bayes.
# The model is trained on a balanced dataset that ensures equitable representation across different particle categories (for roundness, elongation, and rugosity).


############################### Import the essential libraries ######################
import os
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from skimage import measure
from skimage.io import imread
from skimage.measure import regionprops, label
from scipy.stats import skew, kurtosis
from imblearn.over_sampling import SMOTE
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.utils import resample
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout


############## Constants ##########################
IMAGE_DIR = r'C:\Users\Documents\gsd'
SAVE_DIRECTORY = r'C:\Users\Documents'
SEGMENTED_IMAGE_DIR = r'C:\Users\Documents\grain_masks_processed_1'

PIXELS_PER_MM = 900
CALIBRATION_FACTOR = 1 / PIXELS_PER_MM
features = ['Roundness', 'Elongation', 'Rugosity']

IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 1024, 1024, 3

############## Data Loading with Preprocessing ##############
def load_data(image_path):
    images = []
    masks = []
    for file in os.listdir(image_path):
        if file.endswith('.tif') or file.endswith('.png'):
            img = cv2.imread(os.path.join(image_path, file), cv2.IMREAD_COLOR)
            if img is None:
                print(f"Error loading image: {file}")
                continue

            # Denoise image if needed
            img = cv2.GaussianBlur(img, (0, 0), sigmaX=2, sigmaY=2)

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT)) / 255.0
            images.append(img)

            gray = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
            _, mask = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
            mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT)) / 255.0
            masks.append(mask)

    return np.array(images), np.array(masks)

# Use IMAGE_DIR instead of undefined data_path
images, masks = load_data(IMAGE_DIR)
print(f"Loaded {len(images)} images and {len(masks)} masks")

if len(images) == 0 or len(masks) == 0:
    raise ValueError("No images or masks loaded. Check the input directory and image files.")

masks = np.expand_dims(masks, axis=-1)

############## Split Data ##############
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

############## Build U-Net segmentation model ################
def build_unet():
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = UpSampling2D((2, 2))(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = UpSampling2D((2, 2))(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

model = build_unet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=4,
    epochs=10)

preds = model.predict(X_val)


############################# Feature extraction and grain size and shape properties measurement ##################@###

def extract_particle_properties(image_path, calibration_factor):
    image = imread(image_path, as_gray=True)
    binary_image = image > 0.5
    cleared_particles = label(binary_image)
    image_area_mm2 = image.shape[0] * image.shape[1] * (calibration_factor ** 2)
    return cleared_particles, image_area_mm2

def classify_sorting(coefficient_of_variation):
    if coefficient_of_variation < 0.35:
        return 'Very Well Sorted'
    elif coefficient_of_variation < 0.5:
        return 'Well Sorted'
    elif coefficient_of_variation < 0.71:
        return 'Moderately Sorted'
    elif coefficient_of_variation < 2.0:
        return 'Poorly Sorted'
    else:
        return 'Very Poorly Sorted'

def process_directory(directory, save_dir, calibration_factor):
    all_data = {}
    summary_data = []

    for filename in os.listdir(directory):
        if filename.endswith(('.tif', '.jpg', '.png')):
            image_path = os.path.join(directory, filename)
            cleared_particles, total_image_area_mm2 = extract_particle_properties(image_path, calibration_factor)
            props = regionprops(cleared_particles)

########### Collect properties for each particle ##########################
            particle_data = {
                "Label": [],
                "Perimeter (mm)": [],
                "Feret's Diameter (mm)": [],
                "Roundness": [],
                "Rugosity": [],
                "Elongation": [],
                "Particle Size (mm)": []
            }

            for prop in props:
                particle_data["Label"].append(prop.label)
                particle_data["Perimeter (mm)"].append(prop.perimeter * calibration_factor)
                particle_data["Feret's Diameter (mm)"].append(prop.equivalent_diameter * calibration_factor)
                particle_data["Roundness"].append(
                    4 * np.pi * (prop.area * calibration_factor ** 2) / (prop.perimeter * calibration_factor) ** 2
                )
                particle_data["Rugosity"].append(
                    2 * np.sqrt(np.pi * (prop.area * calibration_factor ** 2)) / (prop.perimeter * calibration_factor)
                )
                particle_data["Elongation"].append(prop.major_axis_length / prop.minor_axis_length)
                particle_data["Particle Size (mm)"].append(np.sqrt(prop.area * calibration_factor ** 2))

            sheet_name = os.path.splitext(filename)[0]
            df = pd.DataFrame(particle_data)
            all_data[sheet_name] = df


##################### Summary statistics ########################
            total_particle_area_mm2 = sum([prop.area * (calibration_factor ** 2) for prop in props])
            percentage_area_occupied = (total_particle_area_mm2 / total_image_area_mm2) * 100

            particle_size_mm_series = pd.Series(particle_data["Particle Size (mm)"]).dropna()
            coefficient_of_variation = (
                particle_size_mm_series.std() / particle_size_mm_series.mean()
                if not particle_size_mm_series.empty and particle_size_mm_series.mean() != 0 else 0
            )

            summary_data.append({
                "Image Name": sheet_name,
                "Number of Particles": len(props),
                "Percentage Area Occupied": f"{percentage_area_occupied:.2f}%",
                "Mean Particle Size (mm)": particle_size_mm_series.mean() if not particle_size_mm_series.empty else 0,
                "Coefficient of Variation": coefficient_of_variation,
                "Degree of Sorting": classify_sorting(coefficient_of_variation)
            })

    save_path = os.path.join(save_dir, 'particle_properties_all_images.xlsx')
    with pd.ExcelWriter(save_path, engine='openpyxl') as writer:
        for sheet_name, df in all_data.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
        pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)
    print("Particle properties saved to:", save_path)
    return save_path

#################### Classify the grains based on the degree of roundness, rugosity, and elongation ##############################
def apply_classifications(df):

    def classify_roundness(val):
        if val < 0.25:
            return 'Angular'
        elif 0.25 <= val < 0.5:
            return 'Sub-angular'
        elif 0.5 <= val < 0.75:
            return 'Sub-rounded'
        else:
            return 'Rounded'

    df['Roundness Category'] = df['Roundness'].apply(classify_roundness)

    def classify_rugosity(val):
        if val < 0.25:
            return 'Low'
        elif 0.25 <= val < 0.5:
            return 'Moderate'
        elif 0.5 <= val < 0.75:
            return 'High'
        else:
            return 'Very High'

    df['Rugosity Category'] = df['Rugosity'].apply(classify_rugosity)

    def classify_elongation(val):
        if val < 1.5:
            return 'Equant'
        elif 1.5 <= val < 2.5:
            return 'Slightly Elongated'
        elif 2.5 <= val < 3.5:
            return 'Moderately Elongated'
        else:
            return 'Elongated'

    df['Elongation Category'] = df['Elongation'].apply(classify_elongation)

    return df

def classify_and_save(input_file_path, output_file_path):
    xls = pd.ExcelFile(input_file_path)
    all_sheets_dict = {}

    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        all_sheets_dict[sheet_name] = apply_classifications(df)

    with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
        for sheet_name, df in all_sheets_dict.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    print("Classifications applied and saved to:", output_file_path)
    return output_file_path

def analyze_and_balance(file_path):
    excel_data = pd.read_excel(file_path, sheet_name=None)
    combined_data = pd.concat(excel_data.values(), ignore_index=True)

    categories = ["Roundness Category", "Rugosity Category", "Elongation Category"]
    print("Class Distributions for All Sheets Combined:")
    for category in categories:
        if category in combined_data.columns:
            print(f"\nClass distribution for {category}:")
            print(combined_data[category].value_counts())
        else:
            print(f"\n{category} is not present in the data.")

    def balance_category(data, category, features):
        print(f"\nBalancing the dataset for {category}...")
        target = data[category]
        X = data[features]
        y = target

################# Discard classes with only one sample and apply SMOTE (use of 500 grains per class) ####################
        class_counts = y.value_counts()
        classes_to_keep = class_counts[class_counts > 1].index
        filtered_data = data[data[category].isin(classes_to_keep)]
        X = filtered_data[features]
        y = filtered_data[category]

        print(f"Classes with >1 sample for {category}: {classes_to_keep.tolist()}")
        print("Original class distribution:")
        print(y.value_counts())

        smote = SMOTE(sampling_strategy='not majority', random_state=42)
        X_smote, y_smote = smote.fit_resample(X, y)
        smote_data = pd.DataFrame(X_smote, columns=features)
        smote_data[category] = y_smote

        balanced_data = pd.DataFrame(columns=smote_data.columns)
        for class_label in smote_data[category].unique():
            class_subset = smote_data[smote_data[category] == class_label]
            if len(class_subset) > 500:
                class_subset = resample(class_subset, replace=False, n_samples=500, random_state=42)
            balanced_data = pd.concat([balanced_data, class_subset])

        print("Balanced class distribution:")
        print(balanced_data[category].value_counts())
        return balanced_data

    balanced_roundness = balance_category(combined_data, 'Roundness Category', features)
    balanced_rugosity = balance_category(combined_data, 'Rugosity Category', features)
    balanced_elongation = balance_category(combined_data, 'Elongation Category', features)

    return balanced_roundness, balanced_rugosity, balanced_elongation


########### Create the confusion matrix  and evaluate the models ##################
def save_confusion_matrix(cm, classes, model_name, category_name, save_dir):
    wrapped_classes = ["\n".join(textwrap.wrap(str(label), 10)) for label in classes]
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=wrapped_classes, yticklabels=wrapped_classes, cbar=False,
                annot_kws={"size": 12, "weight": "bold"})
    plt.title(f'{model_name}\nConfusion Matrix for {category_name}', fontsize=14, weight='bold')
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.yticks(rotation=0, fontsize=10)
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f'{model_name}_{category_name}_confusion_matrix.png')
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()
    print(f"Confusion matrix saved to: {save_path}")


def train_and_evaluate_model(X, y, model, model_name, classes, category_name, save_dir):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    print(f"\n{model_name} Performance for {category_name}:")
    print("Accuracy:", accuracy)
    print("Confusion Matrix:\n", cm)
    print("Classification Report:\n", classification_report(y_test, y_pred))
    save_confusion_matrix(cm, classes, model_name, category_name, save_dir)


if __name__ == '__main__':
    properties_file = process_directory(SEGMENTED_IMAGE_DIR, SAVE_DIRECTORY, CALIBRATION_FACTOR)
    classified_file = os.path.join(SAVE_DIRECTORY, 'particle_properties_all_images_classified.xlsx')
    classify_and_save(properties_file, classified_file)
    balanced_roundness, balanced_rugosity, balanced_elongation = analyze_and_balance(classified_file)

    X_roundness = balanced_roundness[features]
    y_roundness = balanced_roundness['Roundness Category']

    X_rugosity = balanced_rugosity[features]
    y_rugosity = balanced_rugosity['Rugosity Category']

    X_elongation = balanced_elongation[features]
    y_elongation = balanced_elongation['Elongation Category']

######################## Define the models (SVM, LogRes, and Naive Bayes) ####################
    models = [
        (SVC(kernel='rbf', random_state=42, C=1.0, probability=True), "Support Vector Machine"),
        (LogisticRegression(max_iter=50, random_state=42, penalty='l2', C=1.0), "Logistic Regression"),
        (GaussianNB(), "Naive Bayes")
    ]

    # Get sorted class labels.
    roundness_classes = sorted(balanced_roundness['Roundness Category'].unique())
    rugosity_classes = sorted(balanced_rugosity['Rugosity Category'].unique())
    elongation_classes = sorted(balanced_elongation['Elongation Category'].unique())
    evaluation_dir = r'C:\Users\skkou\Documents\evaluation'
    for model_obj, model_name in models:
        print("\nClassifying Roundness Category")
        train_and_evaluate_model(X_roundness, y_roundness, model_obj, model_name, roundness_classes, 'Roundness', confusion_save_dir)

        print("\nClassifying Rugosity Category")
        train_and_evaluate_model(X_rugosity, y_rugosity, model_obj, model_name, rugosity_classes, 'Rugosity', confusion_save_dir)

        print("\nClassifying Elongation Category")
        train_and_evaluate_model(X_elongation, y_elongation, model_obj, model_name, elongation_classes, 'Elongation', confusion_save_dir)
